#Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Widget

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("catalog","catalog_ecommerce")
dbutils.widgets.text("schema_source", "bronze")
dbutils.widgets.text("schema_sink", "silver")

#Constantes

In [0]:
catalog         =  dbutils.widgets.get("catalog")
schema_source   =  dbutils.widgets.get("schema_source")
schema_sink     = dbutils.widgets.get("schema_sink")

#Preparacion

In [0]:
df_customers = spark.table(f"{catalog}.{schema_source}.customer")

#Conversion

In [0]:
df_customers_silver = df_customers.filter(col("customer_id").isNotNull()) \
    .filter(col("customer_unique_id").isNotNull()) \
    .withColumn("ingestion_date", current_timestamp()) \
    .dropDuplicates(["customer_id"])

#Guardar tabla

In [0]:
df_customers_silver.write.mode("overwrite").saveAsTable(f"{catalog}.{schema_sink}.customers")